In [12]:
#| default_exp create_thumbnails

# Normalize images and create thumbnails

In [13]:
#| export

import numpy as np
import daft
from PIL import Image

## 1. Normalize

In [14]:
# | export

def _autocontrast_normed(img_arr: np.array) -> np.array:
    """autocontrast from keras - added normalization"""
    x = img_arr.astype(float)
    mean_before = x.mean()
    ## autocontrast from Keras
    x = x - np.min(x)
    x_max = np.max(x)
    if x_max != 0:  x /= x_max
    x *= 255

    ## return to average lightness of input image
    mean_shift = x.mean() - mean_before

    return np.clip(x - mean_shift, 0, 255).astype("uint8")

In [15]:
# | export

@daft.udf(return_dtype=daft.DataType.image(mode="RGB"))
def _autocontrast_img(arrs: daft.Series) -> np.array:
    """apply autocontrast to an image column"""
    return [_autocontrast_normed(img) for img in arrs.to_pylist()]

In [16]:
#| hide
 
@daft.udf(return_dtype=daft.DataType.string())
def _get_image_size_type(imcol: daft.Series):
    """return string of dict of daft image size and type"""
    ims = [Image.fromarray(a) for a in imcol.to_pylist()]
    return [f"Shape:{i.size}, format: {i.format}" for i in ims]

## 2. Create thumbnails

### Notes on thumbnails

- default `cell_size` in [PixPlot is 32](https://github.com/pleonard212/pix-plot/blob/84afbd098f24c5a3ec41219fa849d3eb7b3dc57f/pixplot/pixplot.py#L126). This controls atlas creation.
    - actual shape is set by maximum height being cell_size
    - we should rename to `atlas_cell_height`
- default `lod_cell_height` in [PixPlot is 128](https://github.com/pleonard212/pix-plot/blob/84afbd098f24c5a3ec41219fa849d3eb7b3dc57f/pixplot/pixplot.py#L127). This controls thumbs directory.
    - actual shape is set such that maximum side length is `lod_cell_height`
    - we should rename to `lod_max_side`
- design decision for bedmap
    - we'll resize to max height only, default `thumb_max_height` = 128
    - this will become input to atlas of size `atlas_size` = 16384 (2**14)
        - we'll delay atlas creation until after rasterfairy layout
        - we'll chunk rasterfairy layout into the atlases
    - we'll use [ktx2](https://github.khronos.org/KTX-Software/pyktx/index.html) to store the mipmaps

In [17]:
#| export

def _shape_for_max_side(input_shape: tuple[int],
                       target_max_side: int) -> tuple[int]:
    """
    for a given input shape and a desired max side length,
    return a shape with approximately same aspect ratio satisfying max_side
    """
    w, h = input_shape
    if w >= h:
        new_w = target_max_side
        new_h = int(target_max_side * h / w)
    else:
        new_h = target_max_side
        new_w = int(target_max_side * w / h)
    return (new_w, new_h)

In [18]:
#| export

def _shape_for_max_height(input_shape, target_height: int = 128):
    """
    Given an input shape (width, height) and a target height,
    return the new shape (width, height) with the height set to target_height,
    and the width scaled to preserve aspect ratio. The minimum width is 1.
    """
    w, h = input_shape
    scaled_width = max([int(w / h * target_height),1])
    return (scaled_width, target_height)

In [19]:
#| export

@daft.udf(return_dtype= daft.DataType.image(mode="RGB"))
def _resize_to_max_height(arrs: daft.Series, height: int = 128) -> np.array:
    """resize each image to max height"""
    arrs = arrs.to_pylist()
    input_shapes = [a.shape[:2][::-1] for a in arrs]
    thumb_shapes = [_shape_for_max_height(s, height) for s in input_shapes]
    return [np.array(Image.fromarray(a).resize(s)) for (a, s) in zip(arrs, thumb_shapes)]

In [20]:
#| hide

## i would like to store the thumbnail width.
## However, it looks like you can't put an image in a struct

@daft.udf(return_dtype=daft.DataType.struct({"im":daft.DataType.image(mode="RGB")}))
def _random_image_struct(arrs: daft.Series) -> list[dict[str,np.array]]:
    shapes = arrs.to_pylist()
    return [{"im": (120*np.random.random((*s, 3))).astype(dtype=np.uint8)} for s in shapes]

## The below will panic with a "not implemented" error
# df = daft.from_pydict({"some_shapes": [(127,202), (151,52), (36,405)]})
# df = df.with_column("random_images", _random_image_struct(df["some_shapes"]))
# df.collect()

In [21]:
#| export

def create_thumbnails(df: daft.DataFrame, image_col: str="img", height=128) -> daft.DataFrame:
    """create thumbnail column"""
    df = df.with_column("img_ac", _autocontrast_img(df[image_col]))
    df = df.with_column("thumb", _resize_to_max_height(df["img_ac"], height))
    return df.exclude("img_ac")

In [22]:
#| hide

import nbdev; nbdev.nbdev_export()